# 04. Creación de grafo de las estaciones del sistema metro de CDMX

Carga de librerías

In [23]:
import json
from datetime import datetime, timedelta
import networkx as nx
import plotly.graph_objects as go

Lectura de archivo json con los tiempos de traslado entre estaciones consecutivas

In [24]:
json_file = "../output_metro/travel_times_metro.json"
with open(json_file) as input_json:
    dict_times_metro = json.load(input_json)

Convertimos los datos de cada estacion a ```timedelta```

In [25]:
travel_times_metro = dict()
location_stations = dict()

for route_id in dict_times_metro.keys():
    stations_data = dict_times_metro[route_id]
    converted_route_times = list(map(lambda x: timedelta(hours=x.hour, minutes=x.minute, seconds=x.second), [datetime.strptime(stat_time[1][4:], "%HH%MM%SS") for stat_time in stations_data]))
    
    travel_times_metro[route_id] = [(stations_data[n][0], converted_diff_time) for n, converted_diff_time in enumerate(converted_route_times)]

    for station_data in stations_data:
        location_stations[station_data[0]] = tuple(station_data[2:])

In [26]:
metro_network = dict()

route_ids = travel_times_metro.keys()

for route_id in route_ids:
    stations, travel_times = zip(*travel_times_metro[route_id])
    NumStations = len(stations)
    for i in range(NumStations):
        if stations[i] not in metro_network.keys():
            if i != NumStations - 1:
                metro_network[stations[i]] = [(stations[i+1], travel_times[i+1])]
            else:
                metro_network[stations[i]] = [(stations[i-1], travel_times[i-1])]
        else:
            if i != NumStations - 1:
                metro_network[stations[i]].append((stations[i+1], travel_times[i+1]))
            else:
                metro_network[stations[i]].append((stations[i-1], travel_times[i-1]))

for node, connections in list(metro_network.items()):
    for target, time in connections:
        if target not in metro_network:
            metro_network[target] = []
        nodes_list, _ = zip(*metro_network[target])
        if node not in nodes_list:
            metro_network[target].append((node, time))

In [27]:
G = nx.Graph()

for node, edges in metro_network.items():
    for edge in edges:
        target, weight = edge
        G.add_edge(node, target, weight=weight.total_seconds())

node_degrees = dict(G.degree())

max_degree = max(node_degrees.values())
min_degree = min(node_degrees.values())

node_colors = [node_degrees[node] for node in G.nodes()]

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = location_stations[edge[0]]
    x1, y1 = location_stations[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

node_x = []
node_y = []
node_text = []
for node in G.nodes():
    x, y = location_stations[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"{node}\n# de estaciones: {node_degrees[node]}")

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color='gray'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    textposition="top center",
    marker=dict(
        showscale=True,
        colorscale='turbo',
        cmin=min_degree,
        cmax=max_degree,
        color=node_colors,
        size=10,
        colorbar=dict(
            thickness=15,
            title='# de estaciones conectadas',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Visualización de la red de metro de la CDMX',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper")],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)))
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

fig.show()